## Генератор XMLA-партиций (*by Ivan Rychkov for Kernitsky team*) ❤
---

### 1. Задаём параметры
- Впиши сюда свои параметры генерации партиций и нажми Shift + Enter.

In [15]:
import pandas as pd

# Таблица (группа мер)
table = 'rep70061_fct_transaction'
# База (среда)
db = 'prod_v_rep'
# Поле, по которому делается срез для партиции
slice_column = 'transaction_dt'
# Дата начала первой партиции
date_begin = '2019-01-01'
# Последняя дата
date_end = '2021-12-31'
# Размер партиции: month или day - по месяцам или по дням
partition_size = 'month'

### 2. Запусти ячейку ниже
- **Код здесь менять не нужно**.
- Сгенерированные партиции появятся ниже и в файле output.txt во вкладках меню слева. Их можно отправить себе на внутреннюю почту и добавить в куб.

In [18]:
def strftime(date: pd.Timestamp):
    """Форматирует объект Timestamp в строку."""
    return date.strftime('%Y-%m-%d')

PARTITION_SIZES = {'day', 'month'}
day = pd.Timedelta(1, 'day')

def execute(partition_size,
            date_begin,
            date_end,
            db,
            table,
            slice_column):
    """Генерирует код партиций."""
    # Проверка на верность ввода
    assert partition_size in PARTITION_SIZES, f'Неверно указан partition_size. Возможные значения: {partition_sizes}.'

    date_begin, date_end = map(pd.Timestamp, [date_begin, date_end])

    # Если не месяц, сразу задаём размер шага
    if partition_size == 'day':
        timedelta = day

    partitions = []
    start = date_begin

    while start <= date_end:
        # Делаем шаг размером в месяц, если месяц
        if partition_size == 'month':
            timedelta = day * start.days_in_month
        # Задаём конец интервала
        end = start + timedelta - day
        # Записываем начало и конец интервала
        partitions.append((strftime(start), strftime(end)))
        # Начинаем со следующей даты
        start += timedelta

    queries = []

    for start, end in partitions:
        query = f'''select * from {db}.{table}
        where {slice_column} between date '{start}' and date '{end}'\
        '''

        partition = f'''
        <Partition>
            <ID>{table.replace('_', ' ').upper()} {end}</ID>
            <Name>{table.replace('_', ' ').upper()} {end}</Name>
            <Source xsi:type="QueryBinding">
                <DataSourceID>DM_OLAP</DataSourceID>
                <QueryDefinition>
                {query}
                </QueryDefinition>
            </Source>
            <StorageMode>Molap</StorageMode>
            <ProcessingMode>Regular</ProcessingMode>
            <ProactiveCaching>
                <SilenceInterval>-PT1S</SilenceInterval>
                <Latency>-PT1S</Latency>
                <SilenceOverrideInterval>-PT1S</SilenceOverrideInterval>
                <Source xsi:type="ProactiveCachingInheritedBinding" />
            </ProactiveCaching>
        </Partition>'''
        queries.append(partition)

    result = '\n'.join(queries)

    return result


##############################################
result = execute(partition_size, date_begin, date_end, db, table, slice_column)
with open(table + '_output.txt', 'w+') as file:
    file.write(result)
print(result)


        <Partition>
            <ID>REP70061 FCT TRANSACTION 2019-01-31</ID>
            <Name>REP70061 FCT TRANSACTION 2019-01-31</Name>
            <Source xsi:type="QueryBinding">
                <DataSourceID>DM_OLAP</DataSourceID>
                <QueryDefinition>
                select * from prod_v_rep.rep70061_fct_transaction
        where transaction_dt between date '2019-01-01' and date '2019-01-31'        
                </QueryDefinition>
            </Source>
            <StorageMode>Molap</StorageMode>
            <ProcessingMode>Regular</ProcessingMode>
            <ProactiveCaching>
                <SilenceInterval>-PT1S</SilenceInterval>
                <Latency>-PT1S</Latency>
                <SilenceOverrideInterval>-PT1S</SilenceOverrideInterval>
                <Source xsi:type="ProactiveCachingInheritedBinding" />
            </ProactiveCaching>
        </Partition>

        <Partition>
            <ID>REP70061 FCT TRANSACTION 2019-02-28</ID>
            <Name>R

---